In [1]:
import pandas as pd

colunas = ['CODIGO', 'PROCESSO', 'EMENTA']

ementas_df = pd.read_csv('../data/ementa_ds.zip', compression='zip', header=0, sep=',', quotechar='"', usecols=colunas)

In [2]:
import re

def lei_valida(lei):
    return not re.match("^[\d\.]+/\d{3}$", lei)

def extrair_leis(texto):
    terms = re.findall('\s(\d*\.?\d+\/\d{2,4})[^\d\/]', texto.lower())
    return filter(lei_valida, list(set(terms)))

In [3]:
def normalizar_lei(lei, ementa):
    (numero, ano) = lei.split('/');
    return '{}/{}'.format(normalizar_numero(numero), normalizar_ano(ano))

def normalizar_numero(numero):
    return int(numero.replace('.', ''))

def normalizar_ano(numero):
    ano = int(numero)
    if (ano < 20):
        return '20' + numero
    if (ano < 100):
        return '19' + numero
    return numero

In [4]:
leis = []

for index, row in ementas_df.iterrows():
    for lei in extrair_leis(row['EMENTA']):
        norm = normalizar_lei(lei, row['EMENTA'])
        leis.append([row['CODIGO'], row['PROCESSO'], norm]);


In [5]:
leis_df = pd.DataFrame(data=leis, columns = ['CODIGO', 'PROCESSO', 'LEI'])

df = pd.merge(ementas_df, leis_df, on=['CODIGO', 'PROCESSO']) #, how='left')

df.head()

,CODIGO,PROCESSO,EMENTA,LEI
0,P0000PB7O0000,00111212620168240000,<br> AGRAVO DE INSTRUMENTO. COMUNICAÇÃO EXTE...,10352/2001
1,P0000PP7N0000,05014737820138240061,<br> APELAÇÃO CÍVEL. AÇÃO INDENIZATÓRIA POR ...,7115/1983
2,P0000Q2GP0000,03003626720148240074,<br> MONITÓRIA. DEMANDA LASTREADA EM NOTAS F...,57/2002
3,P0000Q2GP0000,03003626720148240074,<br> MONITÓRIA. DEMANDA LASTREADA EM NOTAS F...,85/2007
4,P0000Q2GP0000,03003626720148240074,<br> MONITÓRIA. DEMANDA LASTREADA EM NOTAS F...,41/2000


Sem normalizar foram encontrados 1679 leis únicas

In [6]:
df.describe()

,CODIGO,PROCESSO,EMENTA,LEI
count,29027,29027,29027,29027
unique,18265,18039,16880,1011
top,P0000P7P70000,00012656420128240166,<br> APELAÇÃO CÍVEL - RECURSO REPETITIVO - R...,6194/1974
freq,27,27,50,2541


In [7]:
groupby = df.groupby('LEI')

groupby[['LEI']].count().head()

,LEI
LEI,
1/1972,1
1/1975,54
1/2006,1
1/2007,2
1/2009,1


In [8]:
c = groupby.filter(lambda x: len(x) > 1000)

c.head()

,CODIGO,PROCESSO,EMENTA,LEI
10,P0000P4FM0000,00065490820138240008,<br> REVISÃO DE CLÁUSULA DE CONTRATO DE PLAN...,8078/1990
42,P0000I22G0000,01526333120158240000,<br> AGRAVO DE INSTRUMENTO. AÇÃO DE INDENIZA...,1060/1950
43,P0000I22G0000,01526333120158240000,<br> AGRAVO DE INSTRUMENTO. AÇÃO INDENIZATÓR...,1060/1950
44,P0000POMN0000,06003188320148240135,<br> APELAÇÃO CÍVEL AÇÃO DE COBRANÇA. SEGURO...,340/2006
45,P0000POMN0000,06003188320148240135,<br> APELAÇÃO CÍVEL AÇÃO DE COBRANÇA. SEGURO...,6194/1974


In [9]:
df.to_csv('../data/leis.csv')
